# Guided Studies into Financial Management
## Index Revisions and Stock Returns

### Colaborators
Dennis Blaufuss,
Lars Wrede,
Nicolas Kepper,
Sophie Merl,
Philipp Voit

### Instructor
Dr. Stefan Scharnoski

### Summary 
HIER MÜSSEN WIR NOCH EINE ZUSAMMENFASSUNG DER ERGEBNISSE SCHREIBEN - WIE ABSTRACT

---

## Table of Content:
1. [Data Proprocessing](#0-bullet)
* [Import Data](#first-bullet)
* [Calculate Daily Returns](#2-bullet)
* [Data Quality Checks](#3-bullet)
* [Descriptive Statistics](#4-bullet)
2. [Price Pressure](#5-bullet)
3. [Investor Attention](#6-bullet)
4. [Systematic Risk and Liquidity](#7-bullet)
* [Systematic Risk - Included Stocks](#8-bullet)
* [Systematic Risk - Excluded Stocks](#9-bullet)
* [Systematic Risk - DAX 30 to DAX 40](#10-bullet)
___

In [ ]:
import pandas as pd
import numpy as np 
import datetime

import yfinance as yf

import performanceanalytics.table.table as pat
from performanceanalytics.charts import performance_summary
import statistics
import statsmodels.formula.api as smf

# 1. Data Preprocessing <a class="anchor" id="0-bullet"></a>

## Import Data <a class="anchor" id="first-bullet"></a>

In [ ]:
''' Import all relevant data.
Parameters
----------
:DAX_aufgenommen: df
    All stocks included in the DAX from 2010 until 2021.
:DAX_ausgeschieden: df
    All stocks excluded from the DAX from 2010 until 2021.
:stock_data: df
    Contains stock ticker as well as names of the stocks of all DAX stock from 22-03-01.
:benchmark:  df
    The MSCI Germany Index was used as a proxy for the market portfolio. 
    This index contains a large number of M-DAX and DAX stocks and is therefore more 
    broadly structured than the DAX. However, the high weight of the DAX shares in the index is problematic, 
    so that it is to be expected that the actual influence is underestimated.
:index_compositions: df
    Contains the deletions/ additions as well as date of change/ announcements & Merger/Spin-Off Information.
-------
'''
DAX_included = pd.read_csv('DAX_included_2010-2021.csv', sep = ';')
DAX_excluded = pd.read_csv('DAX_excluded_2010-2021.csv', sep = ';')
stock_data = pd.read_csv('Companies_Ticker.csv', sep = ';')
benchmark = pd.read_csv('DAX_Kurs.csv', sep = ';')
#index_compositions = pd.read_csv('Historical_Index_Compositions.csv', sep = ';')

In [ ]:
''' Pulls time series data for stocks on a daily basis from 2010-01-01 until 2022-03-01.

Parameters
----------
:stock_dict:  dict
    Contains the stock symbols as key and the time series as values.
-------
'''

stock_dict = {}
for s in stock_data['Symbol']: # iterate for every stock indices
    # Retrieve data from Yahoo Finance
    tickerData = yf.Ticker(s)
    # Save historical data 
    stock_dict[s] = yf.download(s, start='2010-01-01', end='2022-03-01', progress=False)
# Concatenate all data
stocks_as_df = pd.concat(stock_dict, axis = 0)

In [ ]:
''' Pulls time series data for stocks on a daily basis from 2009-06-21 until 2022-03-01.

Parameters
----------
:stock_included:  dict
    Contains the stock symbols as key and the time series as values.
-------
'''

stock_included = {}
for s in DAX_included['Symbol']: # iterate for every stock indices
    # Retrieve data from Yahoo Finance
    tickerData = yf.Ticker(s)
    # Save historical data 
    stock_included[s] = yf.download(s, start='2009-06-21', end='2022-03-01', progress=False)
stock_included_as_df = pd.concat(stock_included, axis = 0)

In [ ]:
''' Pulls time series data for stocks on a daily basis from 2009-06-21 until 2022-03-01.

Parameters
----------
:stock_included:  dict
    Contains the stock symbols as key and the time series as values.
-------
'''

stock_excluded = {}
for s in DAX_excluded['Symbol']: # iterate for every stock indices
    # Retrieve data from Yahoo Finance
    tickerData = yf.Ticker(s)
    # Save historical data 
    stock_excluded[s] = yf.download(s, start='2009-06-21', end='2022-03-01', progress=False)
stock_excluded_as_df = pd.concat(stock_excluded, axis = 0)

## Calculate Daily Returns <a class="anchor" id="2-bullet"></a>

In [ ]:
''' Transform daily price data to daily returns.
Parameters
----------
:returns_daily:  dict
    Contains the stock symbols as key and the daily returns as values.
:returns_daily_excluded: dict
    Contains the stock symbols as key and the daily returns as values.
:returns_daily_included: dict
    Contains the stock symbols as key and the daily returns as values.
:benchmark: df
    Contains daily returns from the benchmark.
-------
'''
returns_daily = {}
for s in stock_data['Symbol']:
    returns_daily[s] = stock_dict[s]['Adj Close'].pct_change()
returns_daily_included = {}
for s in DAX_included['Symbol']:
    returns_daily_included[s] = stock_included[s]['Adj Close'].pct_change()
DAX_included['Aufgenommen'] = pd.to_datetime(DAX_included['Aufgenommen'], format='%d.%m.%Y')

returns_daily_excluded = {}
for s in DAX_excluded['Symbol']:
    returns_daily_excluded[s] = stock_excluded[s]['Adj Close'].pct_change()
DAX_excluded['Ausgeschieden'] = pd.to_datetime(DAX_excluded['Ausgeschieden'], format='%d.%m.%Y')
    
benchmark['Umtauschdatum'] = pd.to_datetime(benchmark['Umtauschdatum'], format='%d.%m.%y')
benchmark = pd.DataFrame(benchmark['Schlusskurs'].astype(float).pct_change()).set_index(benchmark['Umtauschdatum'])

## Data Quality Checks <a class="anchor" id="3-bullet"></a>

In [ ]:
'''Check if stocks_as_df contains NA or zeros in Volume & Adjusted Close.

Parameters
----------
:stocks_as_df:  df
    Contains the time series data as one df.
:stocks_as_df_Volume_is_0:  df
    Contains the rows where Volume == 0.
-------
'''

stocks_as_df_has_nan = np.isnan(np.sum(stocks_as_df)) #no NAs

#(stocks_as_df < 0).any()
#(stocks_as_df = 0).any()

stocks_as_df_Volume_is_0 = stocks_as_df.loc[stocks_as_df["Volume"] == 0] #2774 times 0
stocks_as_df_AdjClose_is_0 = stocks_as_df.loc[stocks_as_df["Adj Close"] == 0] #never 0

In [ ]:
'''Interpolate Volume using 'spline'.

Parameters
----------
:stock_dict_replaced:  dict
    Contains the time series data as stock_dict but with interpolated values for 'Volume'.
-------
'''
stock_dict_replaced = stock_dict
for key in stock_dict_replaced:
    stock_dict_replaced[key].loc[stock_dict_replaced[key]['Volume'] == 0, 'Volume'] = np.nan
    stock_dict_replaced[key]['Volume'].interpolate(method ='spline', order = 2, inplace=True)

#stock_dict_replaced['ZAL.DE'] #stock_dict 2014-10-06 should now have an interpolated value

In [ ]:
'''Check if Adj Close in stocks_as_df differs from previous/ following day.

Parameters
----------
:stocks_as_df:  df
    Contains the time series data as one df.
:stocks_as_df_adjclose_peak_bottom:  dataframe
    Contains the rows where Adj. Close differs
-------
'''
stocks_as_df_adjclose_peak_bottom_list = []
n = 1

while n < len(stocks_as_df)-1:
    if abs(stocks_as_df['Adj Close'][n] -
           statistics.mean([stocks_as_df['Adj Close'][n-1],
                            stocks_as_df['Adj Close'][n+1]])) > .5 * stocks_as_df['Adj Close'][n]:
        stocks_as_df_adjclose_peak_bottom_list.append(stocks_as_df.iloc[n])

    n += 1

stocks_as_df_adjclose_peak_bottom = pd.DataFrame(stocks_as_df_adjclose_peak_bottom_list) #29 times 

## Descriptive Statistics of the whole Dataset <a class="anchor" id="4-bullet"></a>

In [ ]:
''' Calculating measures of location, statistical dispersion and shape.
Parameters
----------
:des_stat:  dataframe
    Contains the descriptive statistics.
-------
'''

des_stat = pd.DataFrame(columns=stock_data['Symbol'], 
                        index=['Observations', 'NAs', 'Minimum', 'Quartile 1', 'Median', 
                               'Artithmetic Mean', 'Geometric Mean', 'Quartile 3', 'Maximum', 'SE Mean',
                               'LCL Mean (.95)', 'UCL Mean (.95)', 'Variance', 'Stdev', 'Skewness','Kurtosis'])

for s in stock_data['Symbol']:
    df = pd.DataFrame(returns_daily[s])
    des_stat[s] = pat.stats_table(df, manager_col=0)
des_stat

In [ ]:
''' Calculating the downside statistics.
Parameters
----------
:down_stat:  dataframe
    Contains the downside statistics.
-------
'''
#down_stat = pd.DataFrame(columns=stock_data['Symbol'], 
#                        index=['Semi Deviation', 'Gain Deviation', 'Loss Deviation', 'Downside Deviation (MAR=2.0%)',
#                               'Downside Deviation (rf=0.5%)', 'Downside Deviation (0%)', 'Maximum Drawdown', 
#                               'Historical VaR (95%)', 'Historical ES (95%)', 'Modified VaR (95%)', 'Modified ES (95%)'])

#for s in stock_data['Symbol']:
#    df = pd.DataFrame(returns_daily[s])
#    down_stat[s] = pat.create_downside_table(df,0)
#down_stat

___
## 2. Price Pressure <a class="anchor" id="5-bullet"></a>

In [ ]:
# Put all dicts into a single one
stock_dict = {**stock_dict, **stock_included, **stock_excluded}

In [ ]:
from datetime import timedelta
from datetime import datetime


def calc_mvr_multiple_days(year_start, year_end, inclusions, day_range, stocks, market_symbol):
    """
    Calculates MVR measures and takes the average over multiple days.
        
    :param year_start:
        year in which to start checking for inclusions to the index
    :param year_end:
        year in which to end checking for inclusions to the index
    :param inclusions:
        dataframe showing inclusions to an index
        Needed columns:
            Aufgenommen - Date included to the index
            Symbol - stock's symbol
    :param day_range:
        range of days for "event period" i
    :param stock:
        stock data for the entire time horizon
    :param market_symbol:
        stock symbol for the market for the entire time horizon
        
    :return:
        mean, stddev, N of all Volume Ratios of included stocks in the selected time period
    """
    s = pd.Series([], dtype='float64')
    for i in day_range:
        s_, _, _, n = calc_mvr(year_start, year_end, inclusions, i, stocks, market_symbol)
        s.append(s_)
    return s, s.mean(), s.std(), n
    

def calc_mvr(year_start, year_end, inclusions, day, stocks, market_symbol):
    """
    MVR of stock i is the mean VR (Volume Ratio) for event period t
    VR of stock i is the ratio of Volume of stock i traded in period t to volume traded in the market times volume traded 
        in the market over the past 8 weeks over volume of stock i in the past 8 weeks
        
    :param year_start:
        year in which to start checking for inclusions to the index
    :param year_end:
        year in which to end checking for inclusions to the index
    :param inclusions:
        dataframe showing inclusions to an index
        Needed columns:
            Aufgenommen - Date included to the index
            Symbol - stock's symbol
    :param day:
        how many days after the announcement date is the "event period" i?
    :param stock:
        stock data for the entire time horizon
    :param market_symbol:
        stock symbol for the market for the entire time horizon
        
    :return:
        series (the series of VRs), mean, stddev, N of all Volume Ratios of included stocks in the selected time period
    """
    
    year_start = datetime.strptime('01.01.' + str(year_start), '%d.%m.%Y')
    year_end = datetime.strptime('31.12.' + str(year_end), '%d.%m.%Y')
    
    inclusions_in_time_period = inclusions[(inclusions['Aufgenommen'] >= year_start) & (inclusions['Aufgenommen'] <= year_end)]
    
    apply_calc_vr = lambda row: calc_vr(row['Aufgenommen'], day, stocks[row['Symbol']], stocks[market_symbol])
    vr_series = inclusions_in_time_period.apply(apply_calc_vr, axis=1)
    return vr_series, vr_series.mean(), vr_series.std(), vr_series.size
    
    
def calc_vr(announcement_date, day, stock, market):
    """
    VR of stock i is the ratio of Volume of stock i traded in period t to volume traded in the market times volume traded 
        in the market over the past 8 weeks over volume of stock i in the past 8 weeks
    $ VR_{it} = \frac{V_{it}}{V_{mt}} * \frac{V_m}{V_i} $
    
    :param announcement_date:
        the announcement date
    :param day:
        how many days after the announcement date is the "event period" i?
    :param stock:
        stock data for the entire time horizon
    :param market:
        stock data for the market for the entire time horizon
    """
    
    start_date = announcement_date - timedelta(weeks=8)
    i = announcement_date + timedelta(days=day)
    
    i_slice = stock[(stock.index >= start_date) & (stock.index <= i)]
    m_slice = market[(market.index >= start_date) & (market.index <= i)]
    
    v_it = stock[stock.index == i]['Volume'].values[0]
    v_mt = market[market.index == i]['Volume'].values[0]
    
    v_i = np.mean(i_slice['Volume'])
    v_m = np.mean(m_slice['Volume'])
    
    return (v_it / v_mt) * (v_m / v_i)

In [ ]:
calc_mvr_multiple_days(2010, 2013, DAX_included,  range(1, 6), stock_dict, '^GDAXI')

In [ ]:
import scipy.stats
year_ranges = [(2010, 2021), (2010, 2015), (2016, 2021)]
year_ranges += [(x, x) for x in range(2010, 2022)]

df = pd.DataFrame(columns=['N', 'MVR Day 1', 'STD Day 1', 't Day 1', '% > 1 Day 1', 'MVR Day 1-5', 'STD Day 1-5', 't Day 1-5', '% > 1 Day 1-5'])

for (y1, y2) in year_ranges:
    s, m, stdev, n = calc_mvr(y1, y2, DAX_included, 1, stock_dict, '^GDAXI')
    t = scipy.stats.testt_1samp(series, 1)
    p = s.gt(1).sum() / s.size
    d = {'N': n, 'MVR Day 1': m, 'STD Day 1': stdev, 't Day 1': t, '% > 1 Day 1': p}
    s, m, stdev, _ = calc_mvr_multiple_days(y1, y2, DAX_included, range(1, 6), stock_dict, '^GDAXI')
    d['MVR Day 1-5'] = m
    d['STD Day 1-5'] = stdev
    d['t Day 1-5'] = scipy.stats.testt_1samp(s, 1)
    d['% > 1 Day 1-5'] = s.gt(1).sum() / s.size
    df = df.append(pd.DataFrame(data=d, index=[str(y1) if y1 == y2 else str(y1) + '-' + str(y2)]))

df

__To-Do's__
* Welche Index Inklusion Effekte gibt es.
* Gibt es einen Pre-announcement drift? 
* Gibt es sonstige Announcement Effekte?

__Paper 1 EMH vs. PPH nachbauen__
* Alle Aktien für die Analyse zusammen suchen - bisher sind nur die neusten 40 enthalten (Sophie hat hier schon eine CSV vorbereitet).
*    Analyse excess return & trading volume on the first 5 days with the cross sectional means.
*    Analysis before and after annoucment as well as after the inclusion day

* Vorschläge hierzu von Stefan:
    * Alles Herausnahmen und Hereinnahmen in den DAX zusammennehmen, da nur so statistische Test möglich sind
    * Bspw. vor 2010, nach 2010 Veränderungen anschauen





Änderungsvorschläge von Stefan
	•	Alles Änderungen zusammen (30 auf 40)
--> inferenzen, statistischen tests nur so möglich
	•	Vor 2010, nach 2010
	•	Überlegen, wie wir das empirisch machen --> counterfactual
	•	Abnormal returns, worauf basiert es; was ist expected return; komplexeres modell
	•	Index Inklusion Effekte
	•	Freiheitsgrade im empirischen Ansatz, solange die Frage beantwortet wird wie sich Inklusion auf Expected Returns und andere Metriken auswirkt
	•	Abnormal returns; Volatilität; Handelsvolumen (ETFs müssen sie nun auch handeln); Investors attention (das sollten wir absprechen; ggf. Aufnahme in Index als Maß für Attention; dazu: googeln)
	•	Korrelationen: Wie ändern sich Korrelation (das könnte attention sein) --> ökonomisch bedeutsam, weil systematisches Risiko
	•	Ggf. Datenfiltern; Daten Fehler; für Preise funktioniert gut; Handelsvolumen nicht so zuverlässig für damals
	•	Wichtig, dass Tage stimmen, ansonsten problematisch
	•	Announcement day & inclusion day
	•	Pre-announcement drift
	•	Announcement Effekte
	•	Dividends announcements --> Literatur

___
## 3. Correlation Analysis <a class="anchor" id="6-bullet"></a>

__To-Do's__ 
* Welche Index Inklusion Effekte gibt es.
* Gibt es einen Pre-announcement drift? 
* Gibt es sonstige Announcement Effekte?

__Paper 2 Investor Attention am Ende nachbauen__
* Korrelationen: Wie ändern sich Korrelation (das könnte attention sein) --> ökonomisch bedeutsam, weil systematisches Risiko --> Hinführung zum 4. Teil

In [ ]:
""" Calculating the change in correlation between each stock and the index itself before and after introduction to it.
The Dates will be standardized (as of t = 0 is the inclusion day) to better visualize.
The correlation will be calculated based on the last 10 days


Time Horizon
----------
Start: 2009-06-21
End: 2022-03-01
----------


"""
data = returns_daily_included
dax = returns_daily['^GDAXI']

for s in DAX_included['Symbol']:
    temp_date = DAX_included.loc[DAX_included['Symbol'] == s]['Aufgenommen'].values[0]
    temp_data_before = data[s].to_frame().loc[:temp_date]['Adj Close']
    temp_data_after = data[s].to_frame().loc[temp_date:]['Adj Close']
    temp_dax_before = dax.loc[:temp_date]
    temp_dax_after = dax.loc[temp_date:]
    corr_before = temp_data_before.corr(temp_dax_before)
    corr_after = temp_data_after.corr(temp_dax_after)


___
## 4. Systematic Risk and Liquidity <a class="anchor" id="7-bullet"></a>

It is examined whether the inclusion of a share in the DAX affects the systematic risk and the liquidity of the share in question.

## Systematic Risk of all newly Stocks included in the DAX <a class="anchor" id="8-bullet"></a>

In [ ]:
''' Calculating the systmatic risk after the inclusion of the Stocks in the DAX.
To estimate the regression equations, OLS was used in conjunction with a correction procedure (Newey/West) 
for serially correlated error terms. 
This approach leads to test statistics that are robust against autocorrelated and 
heteroskedastic disturbance terms.

Time Horizon
----------
Start: 2009-06-21
End: 2022-03-01
----------

Parameters
----------
:sys_risk:  df
    Stock: Name of the specific stock.
    Rank: Sorted after index weight (ascending).
    Delta: Measures the change in the systematic risk of the share triggered by the inclusion.
    p-Value: The two-tailed p-values for the t-stats of the params.
    R^2: R-squared of the model.
-------
'''
i = 0
j = 1
sys_risk = []
while i in range(0,10):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_included.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_included[DAX_included.iloc[i][1]][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_included.iloc[i][0] + datetime.timedelta(days=365))])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_included.iloc[i][0] + datetime.timedelta(days=365))]
    data['Dummy'] = benchmark['Dummy'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_included.iloc[i][0] + datetime.timedelta(days=365))]
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_included.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
  
while i in range(10,len(returns_daily_included)):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_included.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_included[DAX_included.iloc[i][1]][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01'])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data['Dummy'] = benchmark['Dummy'][str(DAX_included.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_included.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
sys_risk = pd.DataFrame(sys_risk)
sys_risk.append(
        {
            'Stock': r"$\varnothing$",
            r"$\Delta$": sys_risk[r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[r"$R^{2}$"].mean()
        }, ignore_index=True
    )

## Systematic Risk of all newly Stocks excluded in the DAX <a class="anchor" id="9-bullet"></a>

In [ ]:
''' Calculating the systmatic risk after the exclusion of the Stocks in the DAX.
To estimate the regression equations, OLS was used in conjunction with a correction procedure (Newey/West) 
for serially correlated error terms. 
This approach leads to test statistics that are robust against autocorrelated and 
heteroskedastic disturbance terms.

Time Horizon
----------
Start: 2009-06-21
End: 2022-03-01
----------

Parameters
----------
:sys_risk:  df
    Stock: Name of the specific stock.
    Rank: Sorted after index weight (ascending).
    Delta: Measures the change in the systematic risk of the share triggered by the inclusion.
    p-Value: The two-tailed p-values for the t-stats of the params.
    R^2: R-squared of the model.
-------
'''

i = 0
j = 1
sys_risk = []
while i in range(0,7):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_excluded.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_excluded[DAX_excluded.iloc[i][1]][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_excluded.iloc[i][0] + datetime.timedelta(days=365))])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_excluded.iloc[i][0] + datetime.timedelta(days=365))]
    data['Dummy'] = benchmark['Dummy'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):str(DAX_excluded.iloc[i][0] + datetime.timedelta(days=365))]   
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_excluded.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
    
while i in range(7,len(returns_daily_excluded)):
    d = []
    for date in benchmark.index:
        if str(date) < str(DAX_excluded.iloc[i][0]):
            d.append(0)
        else: d.append(1)
    benchmark['Dummy'] = d

    data = pd.DataFrame(returns_daily_excluded[DAX_excluded.iloc[i][1]][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01'])
    data['Benchmark'] = benchmark['Schlusskurs'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data['Dummy'] = benchmark['Dummy'][str(DAX_excluded.iloc[i][0] - datetime.timedelta(days=365)):'2022-03-01']
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': DAX_excluded.iloc[i][1],
            'Rank': j, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    j += 1
    i += 1
sys_risk = pd.DataFrame(sys_risk)
sys_risk.append(
        {
            'Stock': r"$\varnothing$",
            r"$\Delta$": sys_risk[r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[r"$R^{2}$"].mean()            
        }, ignore_index=True
    )

## Systematic Risk of the 10 Stocks from DAX Increase in 2021 <a class="anchor" id="10-bullet"></a>

In [ ]:
''' Creating a list and dictionary with all 10 newly added DAX stocks.
Parameters
----------
:newcomers:  list
    Contains the names of the stocks.
:dax_new: dict
    Contains the daily returns of the 10 new stocks.
-------
'''
newcomers = ['AIR.DE', 'SHL.DE', 'ZAL.DE', 'SY1.DE', 'SRT3.DE',  'POAHY', 'HFG.DE', 'BNR.DE', 'QIA.DE', 'PUM.DE']
dax_new = {new: returns_daily[new] for new in newcomers}

In [ ]:
''' Creating the dummy variable - 0 before the inclusion day (2021-09-20) and 1 thereafter.
Parameters
----------
:benchmark:  df
    Contains daily returns as well as the dummy variable.
-------
'''
d = []
for date in benchmark.index:
    if str(date) < '2021-09-20 00:00:00':
        d.append(0)
    else: d.append(1)
benchmark['Dummy'] = d

In [ ]:
''' Calculating the systmatic risk.
To estimate the regression equations, OLS was used in conjunction with a correction procedure (Newey/West) 
for serially correlated error terms. 
This approach leads to test statistics that are robust against autocorrelated and 
heteroskedastic disturbance terms.

Time Horizon
----------
Start: 1 Year before the inclusion day (2020-09-20)
End: 2022-03-01
----------

Parameters
----------
:sys_risk:  df
    Stock: Name of the specific stock.
    Rank: Sorted after index weight (ascending).
    Delta: Measures the change in the systematic risk of the share triggered by the inclusion.
    p-Value: The two-tailed p-values for the t-stats of the params.
    R^2: R-squared of the model.
-------
'''
i = 1
sys_risk = []
for key in dax_new:
    data = pd.DataFrame(dax_new[key]['2020-09-20':'2022-03-01'])
    data['Benchmark'] = benchmark['Schlusskurs']['2020-09-20':'2022-03-01']
    data['Dummy'] = benchmark['Dummy']['2020-09-20':'2022-03-01']
    stocks_as_df['Volume'][key]['2020-09-20':'2021-09-20']    
    data = data.rename(columns = {'Adj Close': 'y', 'Dummy': 'D', 'Benchmark': 'x'})
    reg = smf.ols('y ~ x + D*x', data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    sys_risk.append(
        {
            'Stock': key,
            'Rank': i, 
            r"$\Delta$": reg.params[3], 
            'p_Value': reg.pvalues[3], 
            r"$R^{2}$": reg.rsquared
        }
    )
    i += 1
sys_risk = pd.DataFrame(sys_risk)
sys_risk.append(
        {
            'Stock': r"$\varnothing$",
            r"$\Delta$": sys_risk[r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[r"$R^{2}$"].mean()
        }, ignore_index=True
    )

In [ ]:
'''Distribution of the shares with a higher unit share in the DAX and all those with a weighting of < 1 %. 
Parameters
----------
:des_stat:  df
    N: Number of stocks sorted after the index weight.
    Mean: Mean systematic risk. 
    R^2: Mean R-squared of the model.
-------
'''
des_stat = []
des_stat.append(
        {
            'N': '1-5',
            r"$\varnothing$": sys_risk[:5][r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[:5][r"$R^{2}$"].mean()
        }
    )
des_stat.append(
        {
            'N': '6-10',
            r"$\varnothing$": sys_risk[5:][r"$\Delta$"].mean(),
            r"$R^{2}$": sys_risk[5:][r"$R^{2}$"].mean()
        }
    )
pd.DataFrame(des_stat)